# Exploring Hacker News Posts

Project Goal: we will compare two different posts from Hacker News, a popular site where user-submitted stories (known as "posts") receive votes and comments, similar to reddit.

The two types of posts we'll explore begin with either `Ask HN` or `Show HN`:
 - Ask HN: users submit Ask HN posts to ask the Hacker News community a specific question.
 - Show HN: users submit Show HN posts to show the Hacker News community a project, product, or just something interesting. 

We'll compare these two types of posts to determine the following:
- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?

The [data set](https://www.kaggle.com/hacker-news/hacker-news-posts) that we are working with have been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

# Introduction
First, we'll import libraries and read the dataset into a list of lists called `hn`:


In [1]:
import csv   # Read in the data.

f = open('hacker_news.csv')
hn = list(csv.reader(f))
hn[:5]      #Display the first five rows of hn

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

# Removing Headers from a List of Lists
In order to analyze the data, we'll remove the row containing the column headers, i.e. the first row.

In [2]:
headers = hn[0]   # Extract the first row of data, and assign it to the variable headers.
hn = hn[1:]       # Remove the first row from hn
print(headers)    # Display headers
hn[:5]            # Display the first five rows of hn to verify

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01'],
 ['10301696',
  'Note by Note: The Making of Steinway L1037 (2007)',
  'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
  '8',
  '2',
  'walterbell',
  '9/30/2015 4:12']]

# Extracting Ask HN and Show HN
We'll create new lists of lists containing just the data for the two titles of `Ask HN` or `Show HN`.
We use the string method `startswith` and `lower` to identify posts that begin with either titles.

In [3]:
# Create three empty lists
ask_posts = []
show_posts = []
other_posts = []

# Loop through each row in hn
for post in hn:
    title = post[1]
    
    if title.lower().startswith('ask hn'):
        ask_posts.append(post)
    elif title.lower().startswith('show hn'):
        show_posts.append(post)
    else:
        other_posts.append(post)
        
# Check the number of posts in ask_posts, show_posts, and other_posts
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


# Calculating the Average Number of Comments for Ask HN and Show HN Posts
Now that we have seperated ask posts and show posts into different lists of lists, let's determine which one receive more comments on average.

In [4]:
# Find the total number of comments in ask posts
total_ask_comments = 0

# Use a for loop to iterate over the ask posts
for post in ask_posts:
    num_comments = post[4]
    num_comments = int(num_comments)
    total_ask_comments = total_ask_comments + num_comments
    # More concisely, total_ask_comments += int(post[4])

# Compute the average number of comments on ask posts
avg_ask_comments = total_ask_comments / len(ask_posts)

print(avg_ask_comments)

14.038417431192661


In [5]:
# Find the total number of comments in show posts
total_show_comments = 0

for post in show_posts:
    num_comments = post[4]
    num_comments = int(num_comments)
    total_show_comments = total_show_comments + num_comments
    # More concisely, total_show_comments += int(post[4])
    
avg_show_comments = total_show_comments / len(show_posts)

print(avg_show_comments)

10.31669535283993


On average, ask posts (14 comments) in our sample received more comments than show posts (10 comments).
Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on the ask posts.

# Finding the Number of Ask Posts and Comments by Hour Created
Next, we'll determine if ask posts created at a certain time are more likely to attract comments.
We'll us the following step to perform the analysis:
1. Calculate the number of ask posts created in each hour of the day, along with the total number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

In [21]:
# 1.
# Calculating the number of ask posts created in each hour of the day, 
# along with the total number of comments received.

# Use `datetime` module to work with the data in the created_at column
# Use `datetime.strptime()` constructor to parse dates stored as strings and return datetime objects

import datetime as dt

result_list = [] # A list of lists

for post in ask_posts:
    created_at = post[6] 
    num_comments = post[4] 
    num_comments = int(num_comments)
    result_list.append(
        [created_at, num_comments]
    )

counts_by_hour = {}    # contains the number of ask posts created during each hour of the day.
comments_by_hour = {}  # contains the corresponding number of comments ask posts created at each hour received.
date_format = "%m/%d/%Y %H:%M"

for row in result_list:
    date = row[0]
    comment = row[1]
    time = dt.datetime.strptime(date, date_format).strftime("%H")
    if time not in counts_by_hour:
        counts_by_hour[time] = 1
        comments_by_hour[time] = comment 
    else: 
        counts_by_hour[time] += 1 
        comments_by_hour[time] += comment

In [22]:
# Number of ask posts created in each hour of the day:
counts_by_hour

{'09': 45,
 '13': 85,
 '10': 59,
 '14': 107,
 '16': 108,
 '23': 68,
 '12': 73,
 '17': 100,
 '15': 116,
 '21': 109,
 '20': 80,
 '02': 58,
 '18': 109,
 '03': 54,
 '05': 46,
 '19': 110,
 '01': 60,
 '22': 71,
 '08': 48,
 '04': 47,
 '00': 55,
 '06': 44,
 '07': 34,
 '11': 58}

In [23]:
# Number of comments ask posts created at each hour received:
comments_by_hour

{'09': 251,
 '13': 1253,
 '10': 793,
 '14': 1416,
 '16': 1814,
 '23': 543,
 '12': 687,
 '17': 1146,
 '15': 4477,
 '21': 1745,
 '20': 1722,
 '02': 1381,
 '18': 1439,
 '03': 421,
 '05': 464,
 '19': 1188,
 '01': 683,
 '22': 479,
 '08': 492,
 '04': 337,
 '00': 447,
 '06': 397,
 '07': 267,
 '11': 641}

# Calculating the Average Number of Comments for Ask HN Posts by Hour

In [27]:
# Calculating the average number of comments for posts created during each hour of the day:

# Create a list of lists in which 
# the first element is the hour
# the second element is the average number of comments per post
avg_by_hour = [] 

for hr in comments_by_hour:
    avg_by_hour.append(
        [hr, comments_by_hour[hr] / counts_by_hour[hr]]
    )
    
avg_by_hour

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

# Sorting and Printing Values from a List of Lists
To identify the hours with the highest values, we'll sort the list of lists and print the five highest values in a format that's easier to read.

In [30]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
print(swap_avg_by_hour)        

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

sorted_swap

[[5.5777777777777775, '09'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [16.796296296296298, '16'], [7.985294117647059, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [13.20183486238532, '18'], [7.796296296296297, '03'], [10.08695652173913, '05'], [10.8, '19'], [11.383333333333333, '01'], [6.746478873239437, '22'], [10.25, '08'], [7.170212765957447, '04'], [8.127272727272727, '00'], [9.022727272727273, '06'], [7.852941176470588, '07'], [11.051724137931034, '11']]


[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [40]:
print('Top 5 Hours for Ask Posts Comments')

for avg, hr in sorted_swap[:5]:
    print(
        "{}: {:.2f} average comments per post".format(
            dt.datetime.strptime(hr, "%H").strftime("%H:%M"),avg
        )
    )     

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.5% comments per post.    
<br>
According to the [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts), the time zone is Eastern Time in the US, we could also write 15:00 as 3:00 pm est.

# Conclusion
In this project, we analyzed ask posts and show posts of Hacker News and found that of the posts that receive comments:
- Ask posts received more comments on average
- Ask posts created at 3:00pm est time received more comments on average